# Rolling Window Regression
**Gauge Units**: **feet**

**Goals**
- Target column: Predict next gauge level
- Independent variables: 7 gauge levels for each day of that week, either rainfall/discharge average avg or max of that week

### Resources
- [R Article](https://medium.com/making-sense-of-data/time-series-next-value-prediction-using-regression-over-a-rolling-window-228f0acae363)
- [Python Article](https://www.statsmodels.org/dev/examples/notebooks/generated/rolling_ls.html)

**Procedure**
- Data Manipulation– get the df we need!
- Run all linear regressions, random forest, XGB, polynomial if we have time
- Model evaluation and comparison

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns # Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt # Visualization
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
import math

import warnings # Supress warnings 
warnings.filterwarnings('ignore')

from datetime import date

np.random.seed(0)
sns.set()

In [6]:
levels = pd.read_csv("Data/Daily_Max_Vals.csv", index_col=0)
levels.head()

,date,stid,gauge,discharge,rainfall_inches,rainfall_ft
0,2007-10-01,RA101,2.40,23.9,0.0,0.0
1,2007-10-02,RA101,2.39,23.2,0.0,0.0
2,2007-10-03,RA101,2.39,23.2,0.0,0.0
3,2007-10-04,RA101,2.39,23.2,0.0,0.0
4,2007-10-05,RA101,2.42,25.2,0.0,0.0


In [7]:
levels.corr()

,gauge,discharge,rainfall_inches,rainfall_ft
gauge,1.000000,0.964503,0.526547,0.519915
discharge,0.964503,1.000000,0.508548,0.502007
rainfall_inches,0.526547,0.508548,1.000000,0.987850
rainfall_ft,0.519915,0.502007,0.987850,1.000000


# Since discharge is highly correlated with gauge, we'll choose **rainfall_ft** as our secondary predictor variable